In [71]:
import json
config = json.load(open("./config.json"))
import pandas

In [73]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
#         return response
        return pandas.DataFrame([dict(_) for _ in response])

In [42]:
import os


In [75]:
conn = Neo4jConnection(uri=config['GDB_URL'], user=config['GDB_NAME'], pwd=config['GDB_PASS'])


# 先對特定的使用者來進行研究
＊　

### 推薦內容

In [76]:
store_recommendation_query="""
// Store Recommendation
match (u1:User)-[r1:BUY]->(store:MCC)<-[r2:BUY]-(u2:User)-[r3:BUY]->(rstore:MCC) where u1.userId = '7116107829713266300' 
with rstore.storeId as storeId ,sum(toFloat(r1.buy_count)+toFloat(r2.buy_count)+toFloat(r3.buy_count)) as score 
return storeId,score order by score desc

limit 10000
"""

In [77]:
result = conn.query(store_recommendation_query)
result

,storeId,score
0,5311,98889.0
1,5411,84618.0
2,5812,78820.0
3,5541,74375.0
4,5399,69675.0
...,...,...
286,3741,8.0
287,3000,7.0
288,3366,7.0
289,3357,6.0


### 這個使用者的 Log

In [85]:
q = """MATCH (n:User)-[r]-(s)  where n.userId='7116107829713266300' return n.userId,s.storeId,r.buy_count"""
result = conn.query(q)
result

,n.userId,s.storeId,r.buy_count
0,7116107829713266300,7372,2
1,7116107829713266300,7298,1
2,7116107829713266300,7011,1
3,7116107829713266300,5691,1
4,7116107829713266300,4121,2
5,7116107829713266300,5712,1
6,7116107829713266300,5812,5
7,7116107829713266300,5331,3
8,7116107829713266300,5311,1
9,7116107829713266300,5814,2
